library

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from urllib.parse import urlparse, unquote

Scrape documents link and write as CSV the download link. Two separate requests. 

In [ ]:



csv_path = "C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\gcf_projects.csv"


df_urls = pd.read_csv(csv_path)


all_data = []


start_time = time.time()


for index, row in df_urls.iterrows():
    url_start_time = time.time()  
    project_url = row['url']
    project_code = project_url.split('/')[-1]  

    response = requests.get(project_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.select_one('.content-container .view-content > table')

        if table:
            rows = table.find('tbody').find_all('tr') if table.find('tbody') else []
            for row in rows:
                url_extension = row.find('a')['href'] if row.find('a') else 'N/A'
                full_url = "https://www.greenclimate.fund" + url_extension
                name = row.find('a').text.strip() if row.find('a') else 'N/A'
                doc_type = row.find('td', {'data-header': 'Type'}).text.strip() if row.find('td', {'data-header': 'Type'}) else 'N/A'
                date = row.find('td', {'data-header': 'Date'}).text.strip() if row.find('td', {'data-header': 'Date'}) else 'N/A'
                
                all_data.append({
                    'Project Code': project_code,
                    'URL': full_url,
                    'Name': name,
                    'Type': doc_type,
                    'Date': date
                })
    
    
    url_end_time = time.time()
    print(f"Scraped {project_url} in {url_end_time - url_start_time:.2f} seconds.")


df_final = pd.DataFrame(all_data)


total_time = time.time() - start_time
print(f"Total Processing Time: {total_time:.2f} seconds")


df_final.to_csv('C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\gcf_project_details.csv', index=False)




for index, row in df_final.iterrows():
    document_url = row['URL']
    try:
        response = requests.get(document_url, timeout=10)  
        if response.status_code == 200:
            
            soup = BeautifulSoup(response.text, 'html.parser')
            download_link = soup.find('a', class_='btn btn-primary btn-block text-left p-3 mt-6')
            if download_link:
                
                download_url = download_link['href']
                
                df_final.at[index, 'download_url'] = download_url
                print(f"Processed {document_url} - Download URL added.")
            else:
                print(f"No download link found for {document_url}")
        else:
            print(f"Failed to fetch {document_url}, status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request failed for {document_url}, error: {e}")


df_final.to_csv('C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\gcf_project_details_download.csv', index=False)


//TODO : Some projects do not have documentation. Check with Cesare to determine if extracting text directly from the page is necessary. Below are the project IDs with no documentation.

In [5]:
all_data_path = 'C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\gcf_project_details.csv'
df_urls_path = 'C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\gcf_projects.csv'

all_data = pd.read_csv(all_data_path)
df_urls = pd.read_csv(df_urls_path)

codes_not_in_all_data = df_urls[~df_urls['Ref #'].isin(all_data['Project Code'])]['Ref #']


list_of_codes_not_in_all_data = codes_not_in_all_data.tolist()


print(list_of_codes_not_in_all_data)

['FP225', 'FP226', 'FP227', 'FP228', 'FP229', 'FP230', 'FP231', 'SAP034', 'SAP035', 'SAP036', 'SAP037']


Donwload and organize all pdf. Change folder in save path. You need to read CSV file gcf_project_details_download

In [ ]:


csv_path = 'C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\gcf_project_details_download.csv'

df_final = pd.read_csv(csv_path)

save_path = "C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\gcf_docs"

os.makedirs(save_path, exist_ok=True)

for index, row in df_final.iterrows():
    download_url = row['download_url']
    project_code = row['Project Code']
    

    file_name = unquote(urlparse(download_url).path.split('/')[-1])
    save_file_name = f"{project_code}_{file_name}"
    full_path = os.path.join(save_path, save_file_name)
    
    # CHECK IF ALREADY DOWNLOADED THE FILE. ISSUE WHEN DOWNLOADING STOP. 
    if os.path.exists(full_path):
        print(f"{save_file_name} already exists. Skipping download.")
        continue

    try:
        with requests.get(download_url, stream=True) as r:
            r.raise_for_status()
            
            with open(full_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
                    
        print(f"Downloaded {file_name} as {save_file_name}")
    except requests.HTTPError as e:
        print(f"Failed to download {file_name}. HTTP Error: {e}")
    except Exception as e:
        print(f"Failed to download {file_name}. Error: {e}")
